<a href="https://colab.research.google.com/github/ImranRiazChohan/Cat-and-Dog-Classification-CNN-/blob/master/Cat_and_Dog_Classification(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORT SOME USEFULL LIBARARIES


In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os


## **Unzip data using python code:**
**The following python code will use the OS library, giving you access to the file system, and the zipfile library allowing you to unzip the data.**


In [0]:
import zipfile
local_zip = '/content/drive/My Drive/cat-and-dog.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/content')
zip_ref.close()

# **Let's define the directories.**

### **Dataset containing three subsets: a training set with 1,000 samples of each class, a validation set with 500 samples of each class, and a test set with 500 samples of each class.**

In [0]:
base_dir="/content/cat-and-dog"
test_dir="/content/test_set/test_set"
train_dir="/content/training_set/training_set"


In [0]:
test_dir_cat=os.path.join(test_dir,"cats")
test_dir_dog=os.path.join(test_dir,"dogs")

train_dir_cat=os.path.join(train_dir,"cats")
train_dir_dog=os.path.join(train_dir,"dogs")

In [0]:
train_cat_fnames = os.listdir( train_dir_cat )
train_dog_fnames = os.listdir( train_dir_dog )
test_cat_fnames = os.listdir( test_dir_cat )
test_dog_fnames = os.listdir( test_dir_dog )

print(train_cat_fnames[:5])
print(train_dog_fnames[:5])
print(test_cat_fnames[:5])
print(test_dog_fnames[:5]) 

print("Train data Cats: ",len(os.listdir(train_dir_cat)))
print("Train data Dogs: ",len(os.listdir(train_dir_dog)))

print("Test Data Cats: ",len(os.listdir(test_dir_dog)))
print("Test Data Dog: ",len(os.listdir(test_dir_cat))) 

## **Visualizing data:**
**Now, display a batch of 8 cat and 8 dog pictures. You can rerun the cell to see a fresh batch each time:**

In [0]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

pic_index = 0 # Index for iterating over images
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=8

next_cat_pix = [os.path.join(train_dir_cat, fname) 
                for fname in train_cat_fnames[ pic_index-8:pic_index] 
               ]

next_dog_pix = [os.path.join(train_dir_dog, fname) 
                for fname in train_dog_fnames[ pic_index-8:pic_index]
               ]

for i, img_path in enumerate(next_cat_pix+next_dog_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

Create Small Model

In [0]:
def Create_model():
  model=tf.keras.models.Sequential()
  model.add(layers.Conv2D(32,(3,3),activation="relu",input_shape=(150,150,3)))
  model.add(layers.MaxPool2D(2,2))
  model.add(layers.Conv2D(64,(3,3),activation='relu'))
  model.add(layers.MaxPool2D(2,2))
  model.add(layers.Conv2D(128,(3,3),activation='relu'))
  model.add(layers.MaxPool2D(2,2))
  model.add(layers.Conv2D(128,(3,3),activation='relu'))

  model.add(layers.Flatten())
  model.add(layers.Dense(512,activation="relu"))
  model.add(layers.Dense(1,activation="sigmoid"))
  return model

In [0]:
model=Create_model()
model.summary()

### **compile model:**

We train our model with the binary_crossentropy loss, because it's a binary classification problem and our final activation is a sigmoid.We will use the rmsprop optimizer with a learning rate of 0.001. During training, we will want to monitor classification accuracy. Using the RMSprop optimization algorithm is preferable to stochastic gradient descent (SGD), because RMSprop automates learning-rate tuning for us. (Other optimizers, such as Adam and Adagrad, also automatically adapt the learning rate during training, and would work equally well here.)

In [0]:
model.compile(tf.keras.optimizers.Adam(1e-4),tf.keras.losses.binary_crossentropy,metrics=['accuracy'])

# **Image Data Processing**
Data preprocessing:

In keras it can be done via thekeras.preprocessing.image.ImageDataGenerator class using the rescale parameter. This ImageDataGenerator class allows you to instantiate generators of augmented image batches (and their labels) via .flow(data, labels) or .flow_from_directory(directory). These generators can then be used with the Keras model methods that accept data generators as inputs: fit, evaluate_generator, and predict_generator.

In [0]:
train_datagen=ImageDataGenerator(rescale=1.0/255,rotation_range=40,
            width_shift_range=0.2,height_shift_range=0.2,
            shear_range=0.2,zoom_range=0.2,
            horizontal_flip=True,fill_mode="nearest")
test_datagen=ImageDataGenerator(rescale=1.0/255)

In [0]:
train_gen=train_datagen.flow_from_directory(train_dir,batch_size=20,class_mode="binary",target_size=(150,150))
validation_gen=test_datagen.flow_from_directory(test_dir,batch_size=20,class_mode="binary",target_size=(150,150))

## **Train Model**

In [0]:
history=model.fit_generator(train_gen,epochs=20,steps_per_epoch=100,validation_data=validation_gen,validation_steps=20)

In [0]:
model.save("smallmodel.h5")

## **Evaluating Accuracy and Loss for the Model:**

Let's plot the training/validation accuracy and loss as collected during training:

In [0]:
accuracy=history.history["accuracy"]
loss=history.history['loss']
val_accuracy=history.history['val_accuracy']
val_loss=history.history['val_loss']
epochs=range(len(accuracy))

In [0]:
plt.plot(epochs,accuracy)
plt.plot(epochs,val_accuracy)
plt.title("training and validation Accuracy")
plt.show()

In [0]:
plt.plot(epochs,loss)
plt.plot(epochs,val_loss)
plt.title("training and Validation loss")
plt.show()

## Create Large Scale Model

In [0]:
def Create_Large_model():
  model=tf.keras.models.Sequential()
  model.add(layers.Conv2D(32,(3,3),activation="relu",input_shape=(150,150,3)))
  model.add(layers.Conv2D(32,(3,3),activation='relu'))
  model.add(layers.MaxPool2D(2,2))
  model.add(layers.Conv2D(64,(3,3),activation='relu'))
  model.add(layers.Conv2D(64,(3,3),activation='relu'))
  model.add(layers.MaxPool2D(2,2))
  model.add(layers.Conv2D(128,(3,3),activation='relu'))
  model.add(layers.Conv2D(128,(3,3),activation='relu'))
  model.add(layers.MaxPool2D(2,2))
  model.add(layers.Conv2D(256,(3,3),activation='relu'))
  model.add(layers.Conv2D(256,(3,3),activation='relu'))
  model.add(layers.MaxPool2D(2,2))
  model.add(layers.Conv2D(512,(3,3),activation='relu'))
  model.add(layers.MaxPool2D(2,2))
  model.add(layers.Flatten())
  model.add(layers.Dense(512,activation="relu"))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(1,activation="sigmoid"))
  return model

In [0]:
large_model=Create_Large_model()
large_model.summary()

In [0]:
large_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),loss="binary_crossentropy",metrics=['accuracy'])
History=large_model.fit_generator(train_gen,steps_per_epoch=100,epochs=50,validation_data=validation_gen,validation_steps=50)


In [0]:
Accuracy=History.history["accuracy"]
Loss=History.history['loss']
Val_accuracy=History.history['val_accuracy']
Val_loss=History.history['val_loss']
epochs=range(len(Accuracy))

In [0]:
plt.plot(epochs,Accuracy)
plt.plot(epochs,Val_accuracy)
plt.title("training and validation Accuracy")
plt.show()

In [0]:
plt.plot(epochs,Loss)
plt.plot(epochs,Val_loss)
plt.title("training and Validation loss")
plt.show()

In [0]:
large_model.save("model.h5")

In [0]:
from tensorflow.keras.models import load_model
model=load_model("/content/model.h5")
test_image=image.load_img("/content/cat.jpeg",target_size=(150,150))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result=model.predict(test_image)
map=train_gen.class_indices
print(map)

if result[0][0]>0:
    predict='cat'
else:
    predict='dog'    

print("this image is:",predict)